In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import scipy.stats as st
import importlib
import os
import sys
import platform
import qgrid
import datetime
from scipy.interpolate import make_interp_spline, BSpline
import pickle
import random

from numba import njit

from sync_class import SyncPhenosys
from eda_class import SpikesEDA
from behavior_class import BehaviorAnalysis
from sda_class import SpikesSDA
from report_class import SpikesReport


%load_ext autoreload
%autoreload 2

#pd.set_option('display.max_columns', 100)
#pd.set_option('display.max_rows', 100)

%matplotlib notebook
#plt.style.use('ggplot')
#plt.style.use('gadfly.mplstyle')

# %config InlineBackend.figure_format = 'retina'
#plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")


#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:90% !important; }</style>"))

# dark theme



# ============================================================================================================================
# specify file path & sessions

windows_folder = r"C:/Users/User/Google Drive/3 Projekte/Masterarbeit Laborarbeit Neuroscience/1 Data Analysis"
linux_folder = "/home/max/ExpanDrive/Google Drive/3 Projekte/Masterarbeit Laborarbeit Neuroscience/1 Data Analysis"
mac_folder = "/Users/max/Google Drive/3 Projekte/Masterarbeit Laborarbeit Neuroscience/1 Data Analysis"

# list -> [ session_name, [ttl_missing_rows], load_spikes, [spikes_trials_skip] ]
se_li = [('JG14_190621', [1900,1931,1996,2058,2127],True,[(0,6),(215,'end')]),
         ('JG14_190619', [111, 2781],False,[]),
         ('JG14_190626', [1428, 1824, 1838, 2861, 2910, 3089, 3245, 3430, 3443],False,[]),
         ('JG15_190722', [2094, 2574, 2637, 2808, 2831, 3499],False,[]),
         ('JG15_190725', [366,711,1487,1578,1659,2221,2666,2720,2769,2847,3371,3476],False,[]),
         ('JG18a_190814', [405,2621,2693,2770,2959,3015,3029,3038,3048],False,[]),
         ('JG18b_190828', [1744, 2363, 2648, 2701, 2731, 2778,2953,2967],True,[]),
         ]



# ============================================================================================================================
# Load sessions functions

def add_session(session, gamble_side, sync_obj, missing_rows_ttl, combined, trials, behavior, spikes, trials_skip):
    session_dict = {
        'session':session,
        'gamble_side':gamble_side,
        #'sync_obj':sync_obj,
        'missing_rows_ttl':missing_rows_ttl,
        'combined':combined,
        'trials':trials,
        'behavior':behavior,
        'trials_skip':trials_skip,
    }
    if spikes != False:
        session_dict['spikes']=spikes
        
    # add session dict to all sessions dict
    all_sessions_dict[session]=session_dict
    return session_dict


def load_session(session, missing_rows_ttl=[], lo_spikes=False, trials_skip=[]):

    # load calss and set folder depending on platform

    # load calss and set folder depending on platform
    if platform.system() == 'Linux':
        # Linux
        folder = linux_folder + '/' + session
    elif platform.system() == 'Windows':
        # windows
        folder = windows_folder + r"/" + session
    elif platform.system() == 'Darwin':
        folder = mac_folder + r"/" + session

    sync_obj = SyncPhenosys(session, folder, 7, 1, rows_missing_ttl)   
    gamble_side = sync_obj.gamble_side

    # first find missing rows
    #missing_rows_csv = []
    #combined = sync_obj.combine_dataframes(missing_rows_ttl, missing_rows_csv, 'channel 1', sync_obj.csv, False)
    #trials_df = sync_obj.get_trials(combined)
    behavior_obj = BehaviorAnalysis(sync_obj, deselect_trials)
    
    if lo_spikes:
        spikes_obj = SpikeAnalysis(behavior_obj)
    #    spikes = SpikeAnalysis(session, folder, sync_obj)
    #    for a,b in trials_skip:
    #        if b == 'end':
    #            spikes.trials_df.loc[a:,'select'] = False
    #        else:
    #            spikes.trials_df.loc[a:b,'select'] = False
    #else:
    #    spikes = False

    return session, gamble_side, sync_obj, behavior_obj, spikes_obj


def load_all_sessions(se_li):
    all_sessions = dict()
    for session, missing_rows_ttl, lo_spikes, trials_skip in se_li:
        session, gamble_side, sync_obj, combined, trials, behavior, spikes = load_session(session, missing_rows_ttl, lo_spikes, trials_skip)
        
        session_dict = add_session(session, gamble_side, sync_obj, missing_rows_ttl, combined, trials, behavior, spikes, trials_skip)  
        all_sessions[session]=session_dict

    return all_sessions


        
# ============================================================================================================================
# save & load sessions from pickl functions

# save dict to pickl
def save_all_sessions_dict_pickl():
    if platform.system() == 'Linux':
        # Linux
        folder = linux_folder
    elif platform.system() == 'Windows':
        # windows
        folder = windows_folder
    elif platform.system() == 'Darwin':
        folder = mac_folder 
        
    file = folder + '/all_sessions_dict.pkl'
    with open(file, 'wb') as dump:
        pickle.dump(all_sessions_dict, dump)

# load dict from pickl
def load_all_sessions_dict_pickl():
    if platform.system() == 'Linux':
        # Linux
        folder = linux_folder
    elif platform.system() == 'Windows':
        # windows
        folder = windows_folder
    elif platform.system() == 'Darwin':
        folder = mac_folder 
        
    file = folder + '/all_sessions_dict.pkl'
    
    with open(file, 'rb') as dump:
        all_sessions_dict = pickle.load(dump)
    return all_sessions_dict

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## JG14_190621

In [13]:
%%time

session = 'JG14_190621'
folder = mac_folder + r"/" + session
# sync
rows_missing_ttl = [1900,1931,1996,2058,2127]
# behavior
deselect_trials = [(0,6),(215,'end')]
# sda
window = 2000
iterations = 1000
bins = 50

sync_obj = SyncPhenosys(session, folder, 7, 1, rows_missing_ttl) 
behavior_obj = BehaviorAnalysis(sync_obj, deselect_trials)
eda_obj = SpikesEDA(behavior_obj)
sda_obj = SpikesSDA(eda_obj)
sda_obj.spiketimes_data_ar, sda_obj.reward_aligned_ar, sda_obj.binned, sda_obj.mean_ar, sda_obj.percentil_ar = sda_obj.get_bootstrap_all_clusters(window, iterations,bins,'reward')
report_obj = SpikesReport(eda_obj)

JG14_190621 = type('obj', (object,), 
                   {'sync':sync_obj,
                    'behavior':behavior_obj,
                    'eda':eda_obj,
                    'sda': sda_obj,
                    'report': report_obj,
                   })


/opt/anaconda3/envs/spikes/lib/python3.8/site-packages/IPython/core/interactiveshell.py:2382: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = fn(*args, **kwargs)
/opt/anaconda3/envs/spikes/lib/python3.8/site-packages/pandas/core/indexing.py:1719: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/max/Google Drive/3.1 Code Repository/3 Masterarbeit/Spikes/sda_class.py:246: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  reward_aligned_ar[cl,:]=np.array(

CPU times: user 6min 40s, sys: 4.89 s, total: 6min 45s
Wall time: 6min 48s


In [ ]:
%%time

#JG14_190621.eda.generate_plots()
#JG14_190621.sda.generate_plots()
JG14_190621.report.generate_report()

## JG18b_190828

In [10]:
%%time

session = 'JG18b_190828'
folder = mac_folder + r"/" + session
# sync
rows_missing_ttl = [1744, 2363, 2648, 2701, 2731, 2778,2953,2967]
# behavior
deselect_trials = [(0,0),(204,'end')]
# eda
skip_clusters = [116, 280]
# sda
window = 2000
iterations = 1000
bins = 50

sync_obj = SyncPhenosys(session, folder, 7, 1, rows_missing_ttl) 
behavior_obj = BehaviorAnalysis(sync_obj, deselect_trials)
eda_obj = SpikesEDA(behavior_obj)
sda_obj = SpikesSDA(eda_obj)
sda_obj.spiketimes_data_ar, sda_obj.reward_aligned_ar, sda_obj.binned, sda_obj.mean_ar, sda_obj.percentil_ar = sda_obj.get_bootstrap_all_clusters(window, iterations,bins,'reward')
report_obj = SpikesReport(eda_obj)

JG18b_190828 = type('obj', (object,), 
                   {'sync':sync_obj,
                    'behavior':behavior_obj,
                    'eda':eda_obj,
                    'sda': sda_obj,
                    'report': report_obj,
                   })



/opt/anaconda3/envs/spikes/lib/python3.8/site-packages/IPython/core/interactiveshell.py:2382: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = fn(*args, **kwargs)
/opt/anaconda3/envs/spikes/lib/python3.8/site-packages/pandas/core/indexing.py:1719: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/max/Google Drive/3.1 Code Repository/3 Masterarbeit/Spikes/sda_class.py:246: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  reward_aligned_ar[cl,:]=np.array(

CPU times: user 17min 28s, sys: 13.8 s, total: 17min 42s
Wall time: 17min 49s


In [ ]:
%%time

#JG18b_190828.eda.generate_plots()
JG18b_190828.sda.generate_plots(window,iterations,bins)

In [ ]:
%%time
JG18b_190828.report.generate_report()